# Assignment 1: Discrete Selectors

* Make a chropleth map of TotalLifts in each country.  
* Allow users to toggle Snowparks and Night Ski features. Your selector should allow them to filter to ski resorts that have one, or both of these features. If neither is selected the map should display the total lifts in each country.  

In [5]:
import pandas as pd
import plotly.express as px

ski_resorts = pd.read_csv("../data/European_Ski_Resorts.csv").drop("Unnamed: 0", axis=1)

ski_resorts.head()

,Resort,Country,HighestPoint,LowestPoint,DayPassPriceAdult,BeginnerSlope,IntermediateSlope,DifficultSlope,TotalSlope,Snowparks,NightSki,SurfaceLifts,ChairLifts,GondolaLifts,TotalLifts,LiftCapacity,SnowCannons
0,Alpendorf (Ski amedé),Austria,1980,740,52,30,81,4,115,Yes,No,22,16,11,49,75398,600
1,Soldeu-Pas de la Casa/​Grau Roig/​El Tarter/​C...,Andorra,2640,1710,47,100,77,33,210,Yes,Yes,37,28,7,72,99017,1032
2,Oberau (Wildschönau),Austria,1130,900,30,1,0,1,2,No,No,2,0,0,2,1932,0
3,Dachstein West,Austria,1620,780,42,15,33,3,51,Yes,Yes,25,8,3,36,32938,163
4,Rosa Khutor,Southern Russia,2320,940,22,30,26,21,77,Yes,No,6,11,10,27,49228,450


In [29]:
from dash import dcc, html
from jupyter_dash import JupyterDash

from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

import plotly.express as px
import pandas as pd

lift_type_by_country = (
    ski_resorts
    .groupby("Country", as_index=False)
    .agg({"SurfaceLifts":"sum", "ChairLifts":"sum", "GondolaLifts":"sum", "TotalLifts":"sum"})
    .sort_values("TotalLifts", ascending=False)
)

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2(id="title"),
    dcc.Checklist(
        id="Feature Checklist",
        options=["Has Snow Park", "Has Night Ski"],
        value=["Has Snow Park", "Has Night Ski"]
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("Feature Checklist", "value"),
)

def plot_lift_number(features):
    if features == []:
        df = (ski_resorts
              .groupby("Country", as_index=False)
              .agg({"TotalLifts":"sum"})
             )
    elif len(features) == 2:
        df = (ski_resorts
              .query("Snowparks == 'Yes' and NightSki == 'Yes'")
              .groupby("Country", as_index=False)
              .agg({"TotalLifts":"sum"})
             )
    elif features == ["Has Snow Park"]:
        df = (ski_resorts
              .query("Snowparks == 'Yes'")
              .groupby("Country", as_index=False)
              .agg({"TotalLifts":"sum"})
             )
    else:
        df = (ski_resorts
              .query("NightSki == 'Yes'")
              .groupby("Country", as_index=False)
              .agg({"TotalLifts":"sum"})
             )
    fig = px.choropleth(
        df,
        locations="Country",
        color="TotalLifts",
        locationmode="country names",
        scope="europe",
    ).update_geos(fitbounds="locations").update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    

    return fig
    
if __name__ == "__main__":
    app.run_server(
        mode="inline",
        port=2023,
        debug=True,      
)

## Assignment 2: Sliders

* Build a Bar Chart of the total number of ski resorts in each country.
* Embed this chart into a dash app and build a slider that filters ski resorts based on the HighestPoint column. For example, if users slide to 2500 meters, only resorts that have a point higher than 25 meters should be included in the bar chart count.

In [27]:
from dash import dcc, html
from jupyter_dash import JupyterDash

from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

import plotly.express as px
import pandas as pd

ski_resorts = pd.read_csv("../Data/European_Ski_Resorts.csv").drop("Unnamed: 0", axis=1)

app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.Slider(
        id="Elevation Slider",
        min=0,
        max=4000,
        value=500,
        step=500,
        marks={i: f'{i} Meters' for i in range(0, 4001, 500)}
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('graph', 'figure'),
    Input("Elevation Slider", "value"),
)

def plot_lift_number(elevation):
#     if not elevation:
#         raise PreventUpdate
    df=(ski_resorts.query("HighestPoint > @elevation")
        .groupby("Country", as_index=False)
        .agg(Ski_Resorts_by_Country= ("Country", "count"))
    )
    fig = px.bar(
    df,
    x="Country",
    y="Ski_Resorts_by_Country"
    )
    return fig

if __name__ == "__main__":
    app.run_server(
        mode="inline", 
        debug=True,
        port=8681
)

## Assignment 3: Multiple Interactive Elemetns

* Build an application that displays a map of the count of ski resorts by country. 
* Include the toggle for Snow Park and Night Ski that you developed in assignment 1
* Include the slider you developed in Assignment 2
* Your final app should allow users to filter both on highest point and whether or not the ski resort has night skiing and snow parks, and display an accurate count of ski resorts that match the criteria in each country. 

In [2]:
from dash import dcc, html
from jupyter_dash import JupyterDash

from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

import plotly.express as px
import pandas as pd

ski_resorts = pd.read_csv("../Data/European_Ski_Resorts.csv").drop("Unnamed: 0", axis=1)

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2(id="title"),
    dcc.Slider(
        id="Elevation Slider",
        min=0,
        max=4000,
        step=500,
        value=500,
        marks={i: f'{i}' for i in range(0, 4001, 500)}
),
    dcc.Checklist(
        id="Feature Checklist",
        options=["Has Snow Park", "Has Night Ski"], 
        value=["Has Snow Park", "Has Night Ski"], 
),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output('title', 'children'),
    Output('graph', 'figure'),
    Input("Elevation Slider", "value"),
    Input("Feature Checklist", "value"),
)

def plot_lift_number(elevation, features):
#     if not elevation:
#         raise PreventUpdate
        
    title = f"Ski Resorts with Elevation Over {elevation}M Max Elevation"
    
    ski_resorts_filtered = ski_resorts.query("HighestPoint > @elevation")
    
    if features == []:
        df = (ski_resorts_filtered
              .groupby("Country", as_index=False)
              .agg(ResortCount= ("Country", "count"))
             )
    elif len(features)==2:
        df = (ski_resorts_filtered
              .query("Snowparks == 'Yes' and NightSki == 'Yes'")
              .groupby("Country", as_index=False)
              .agg(ResortCount= ("Country", "count"))
             )
    elif features == ["Has Snow Park"]:
        df = (ski_resorts_filtered
              .query("Snowparks == 'Yes'")
              .groupby("Country", as_index=False)
              .agg(ResortCount= ("Country", "count"))
             )
    else:
        df = (ski_resorts_filtered
              .query("NightSki == 'Yes'")
              .groupby("Country", as_index=False)
              .agg(ResortCount= ("Country", "count"))
             )
    
    fig = px.choropleth(
        df,
        locations="Country",
        color="ResortCount",
        locationmode="country names",
        scope="europe"
    ).update_geos(fitbounds="locations").update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return title, fig 
    
if __name__ == "__main__":
    app.run_server(
        mode="inline", 
        debug=True, 
        port=8356
)